# The Predictable and Unpredictable Components of 21st-Century Sea-Level Rise

This notebook implements three core analyses:

1. **Variance decomposition** — Partition total SLR projection uncertainty into three additive components at each time horizon: *constrained* (DOLS parameter uncertainty), *scenario* (across SSP pathways), and *ice sheet dynamics* (residual deep uncertainty).

2. **Hindcast cross-validation** — Calibrate the DOLS model on truncated records (through 2000, 2005, 2010) and evaluate forecast skill against the withheld observations, demonstrating that the "predictable" component is genuinely predictable.

3. **Predictability partition figure** — Visualize how the relative contribution of each uncertainty source evolves from 2030 to 2150, showing the transition from a forecastable near-term to a deeply uncertain long-term dominated by ice sheet dynamics.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import FancyBboxPatch
from scipy import interpolate, stats
import warnings
import sys
import os

# Add project paths
sys.path.insert(0, os.path.dirname(os.path.abspath('__file__')))

from slr_analysis import (
    calibrate_alpha_dols_quadratic,
    resample_to_monthly,
    DOLSQuadraticResult
)
from slr_projections import project_gmsl_ensemble, project_gmsl_from_temperature

warnings.filterwarnings('ignore', category=FutureWarning)

# Plot style
plt.rcParams.update({
    'figure.dpi': 150,
    'font.size': 10,
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'legend.fontsize': 9,
    'figure.facecolor': 'white',
    'axes.facecolor': 'white',
    'axes.grid': True,
    'grid.alpha': 0.3,
})

# Conversion factor
M_TO_MM = 1000.0

print('Imports loaded successfully.')

## 1. Load Processed Data

In [ ]:
# ============================================================
# Load all data from H5 file
# ============================================================
h5_path = '../data/processed/slr_processed_data.h5'

with pd.HDFStore(h5_path, 'r') as store:
    # Configuration
    config = store['/config']
    baseline_start = int(config['baseline_start'].iloc[0])
    baseline_end = int(config['baseline_end'].iloc[0])
    
    # Harmonized observations (baseline = 1995-2005)
    df_frederikse = store['/harmonized/df_frederikse_h']
    df_berkeley = store['/harmonized/df_berkeley_h']
    df_imbie_wais = store['/harmonized/df_imbie_wais_h']
    
    # Temperature projections (IPCC AR6)
    temp_hist = store['/projections/temp/Historical']
    temp_projections = {}
    ssp_keys = {
        'SSP1-1.9': 'SSP1_1_9', 'SSP1-2.6': 'SSP1_2_6',
        'SSP2-4.5': 'SSP2_4_5', 'SSP3-7.0': 'SSP3_7_0',
        'SSP5-8.5': 'SSP5_8_5'
    }
    for name, key in ssp_keys.items():
        temp_projections[name] = store[f'/projections/temp/{key}']
    
    # IPCC GMSL projections (with components)
    ipcc_gmsl = {}
    ssp_gmsl_keys = {
        'SSP1-1.9': 'ssp119', 'SSP1-2.6': 'ssp126',
        'SSP2-4.5': 'ssp245', 'SSP3-7.0': 'ssp370',
        'SSP5-8.5': 'ssp585'
    }
    for name, key in ssp_gmsl_keys.items():
        ipcc_gmsl[name] = store[f'/projections/gmsl/{key}']

print(f'Baseline: {baseline_start}-{baseline_end}')
print(f'Frederikse GMSL: {df_frederikse.index[0].year}-{df_frederikse.index[-1].year} ({len(df_frederikse)} obs)')
print(f'Berkeley Earth: {df_berkeley.index[0].year}-{df_berkeley.index[-1].year} ({len(df_berkeley)} obs)')
print(f'IMBIE WAIS: {df_imbie_wais.index[0].year}-{df_imbie_wais.index[-1].year} ({len(df_imbie_wais)} obs)')
print(f'Temperature scenarios: {list(temp_projections.keys())}')

## 2. DOLS Calibration — Full Record

Fit the quadratic semi-empirical model on the full available record to establish reference coefficients:

$$\text{rate} = \frac{d\alpha}{dT} \cdot T^2 + \alpha_0 \cdot T + \beta$$

In [ ]:
# ============================================================
# Prepare aligned sea level and temperature for calibration
# ============================================================
# Resample annual Frederikse to monthly for alignment with Berkeley
df_fred_monthly = resample_to_monthly(
    df_frederikse, value_col='gmsl', unc_col='gmsl_sigma'
)

# Align on common time range
common_idx = df_fred_monthly.index.intersection(df_berkeley.index)
sl_series = df_fred_monthly.loc[common_idx, 'gmsl']
temp_series = df_berkeley.loc[common_idx, 'temperature']

# Full-record calibration
result_full = calibrate_alpha_dols_quadratic(
    sea_level=sl_series,
    temperature=temp_series,
    n_lags=2
)

print('=== Full-Record DOLS Quadratic Calibration ===')
print(result_full)
print(f'\nRate model (m/yr):')
print(f'  rate = {result_full.dalpha_dT:.6f} T² + {result_full.alpha0:.6f} T + {result_full.trend:.6f}')
print(f'  rate = {result_full.dalpha_dT*M_TO_MM:.3f} T² + {result_full.alpha0*M_TO_MM:.3f} T + {result_full.trend*M_TO_MM:.3f}  [mm/yr]')

## 3. Monte Carlo Ensemble Projections

Generate forward projections under all five SSP scenarios, propagating coefficient uncertainty through Monte Carlo sampling.

In [ ]:
# ============================================================
# Build continuous temperature time series for each SSP
# (Historical + SSP, merged on overlap)
# ============================================================
def build_full_temperature_scenario(temp_hist, temp_ssp, baseline_start, baseline_end):
    """Combine historical and SSP temperature projections into a single series,
    re-baselined to the project baseline period."""
    # Use SSP data from 2015 onward, historical before
    hist_part = temp_hist[temp_hist['decimal_year'] < 2015].copy()
    combined = pd.concat([hist_part, temp_ssp]).sort_index()
    # Remove any duplicates
    combined = combined[~combined.index.duplicated(keep='last')]
    return combined

full_temp_scenarios = {}
for name, temp_ssp in temp_projections.items():
    full_temp_scenarios[name] = build_full_temperature_scenario(
        temp_hist, temp_ssp, baseline_start, baseline_end
    )

print(f'Temperature scenario time ranges:')
for name, df in full_temp_scenarios.items():
    print(f'  {name}: {df.decimal_year.min():.0f}-{df.decimal_year.max():.0f} ({len(df)} years)')

In [ ]:
# ============================================================
# Run ensemble projections
# ============================================================
N_SAMPLES = 2000
BASELINE_YEAR = 2005.0  # IPCC AR6 reference year

# Extract coefficients in [a, b, c] form for projection: rate = a*T² + b*T + c
# Note: the covariance matrix from DOLS is for [coeff(∫T²), coeff(∫T), trend, ...]
# where coeff(∫T²) = dalpha_dT / 2
# We need to transform to get covariance of [dalpha_dT, alpha0, trend]
coeffs_abc = np.array([
    result_full.dalpha_dT,   # a = dα/dT
    result_full.alpha0,       # b = α₀
    result_full.trend         # c = β
])

# Transform covariance: dalpha_dT = 2 * coeff[0]
# So var(dalpha_dT) = 4 * var(coeff[0])
# cov(dalpha_dT, x) = 2 * cov(coeff[0], x)
cov_raw = result_full.covariance[:3, :3]  # first 3 params
transform = np.diag([2.0, 1.0, 1.0])  # dalpha_dT = 2 * coeff[0]
cov_abc = transform @ cov_raw @ transform.T

print(f'Projection coefficients [a, b, c]:')
print(f'  a (dα/dT)  = {coeffs_abc[0]*M_TO_MM:.4f} ± {np.sqrt(cov_abc[0,0])*M_TO_MM:.4f} mm/yr/°C²')
print(f'  b (α₀)    = {coeffs_abc[1]*M_TO_MM:.4f} ± {np.sqrt(cov_abc[1,1])*M_TO_MM:.4f} mm/yr/°C')
print(f'  c (trend)  = {coeffs_abc[2]*M_TO_MM:.4f} ± {np.sqrt(cov_abc[2,2])*M_TO_MM:.4f} mm/yr')

# Run ensemble
ensemble_results = project_gmsl_ensemble(
    coefficients=coeffs_abc,
    coefficients_cov=cov_abc,
    temperature_projections=full_temp_scenarios,
    baseline_year=BASELINE_YEAR,
    baseline_gmsl=0.0,
    n_samples=N_SAMPLES,
    seed=42
)

print(f'\nEnsemble projections complete ({N_SAMPLES} samples per scenario).')
for name, df in ensemble_results['scenarios'].items():
    row_2100 = df[df['decimal_year'] >= 2099].iloc[0] if len(df[df['decimal_year'] >= 2099]) > 0 else None
    if row_2100 is not None:
        print(f'  {name} at ~2100: {row_2100["gmsl"]*M_TO_MM:.0f} [{row_2100["gmsl_lower"]*M_TO_MM:.0f}, {row_2100["gmsl_upper"]*M_TO_MM:.0f}] mm')

## 4. Variance Decomposition

Decompose total SLR projection uncertainty into three components at each time horizon:

- **$\sigma^2_{\text{constrained}}$** : From DOLS coefficient uncertainty, propagated forward for a *fixed* SSP (we use the across-scenario mean to isolate parameter uncertainty)
- **$\sigma^2_{\text{scenario}}$** : Variance of median projections *across* SSP pathways (reflects societal choice)
- **$\sigma^2_{\text{ice}}$** : Residual variance between our observationally-constrained projection and the full IPCC range, dominated by ice sheet dynamics

The decomposition is computed at decadal intervals matching IPCC projection time steps.

In [ ]:
# ============================================================
# 4a. Run full Monte Carlo to get per-scenario uncertainty 
#     at each time step (not just median/5-95)
# ============================================================

def run_full_ensemble(coefficients, cov, temp_scenarios, baseline_year=2005.0,
                      n_samples=2000, seed=42):
    """
    Run Monte Carlo ensemble and return full sample arrays.
    Returns dict of {scenario: {'time': array, 'gmsl_samples': (n_samples, n_time)}}
    """
    rng = np.random.RandomState(seed)
    coeff_samples = rng.multivariate_normal(coefficients, cov, n_samples)
    
    results = {}
    for sname, temp_df in temp_scenarios.items():
        T = temp_df['temperature'].values
        time_years = temp_df['decimal_year'].values
        dt = np.diff(time_years)
        baseline_idx = np.argmin(np.abs(time_years - baseline_year))
        nt = len(T)
        
        gmsl_samples = np.zeros((n_samples, nt))
        rate_samples = np.zeros((n_samples, nt))
        
        for k in range(n_samples):
            a, b, c = coeff_samples[k]
            rate = a * T**2 + b * T + c
            rate_samples[k] = rate
            
            gmsl = np.zeros(nt)
            for i in range(baseline_idx, nt - 1):
                gmsl[i+1] = gmsl[i] + 0.5 * (rate[i] + rate[i+1]) * dt[i]
            for i in range(baseline_idx, 0, -1):
                gmsl[i-1] = gmsl[i] - 0.5 * (rate[i] + rate[i-1]) * dt[i-1]
            gmsl_samples[k] = gmsl
        
        results[sname] = {
            'time': time_years,
            'gmsl_samples': gmsl_samples,
            'rate_samples': rate_samples,
            'gmsl_median': np.median(gmsl_samples, axis=0),
            'gmsl_p5': np.percentile(gmsl_samples, 5, axis=0),
            'gmsl_p95': np.percentile(gmsl_samples, 95, axis=0),
            'gmsl_p17': np.percentile(gmsl_samples, 17, axis=0),
            'gmsl_p83': np.percentile(gmsl_samples, 83, axis=0),
        }
    
    return results, coeff_samples

mc_results, coeff_samples = run_full_ensemble(
    coeffs_abc, cov_abc, full_temp_scenarios, 
    baseline_year=BASELINE_YEAR, n_samples=N_SAMPLES, seed=42
)

print('Full Monte Carlo ensemble complete.')

In [ ]:
# ============================================================
# 4b. Compute variance decomposition at decadal intervals
# ============================================================

# Target years for decomposition (matching IPCC decadal steps)
target_years = np.arange(2020, 2160, 10)

# We use 5 main SSPs for scenario spread
scenario_names = ['SSP1-1.9', 'SSP1-2.6', 'SSP2-4.5', 'SSP3-7.0', 'SSP5-8.5']

decomposition = []

for yr in target_years:
    # ---- σ²_constrained: mean coefficient uncertainty across scenarios ----
    # For each scenario, get the variance of GMSL at this year across MC samples
    var_constrained_per_ssp = []
    median_per_ssp = []
    
    for sname in scenario_names:
        mc = mc_results[sname]
        time = mc['time']
        idx = np.argmin(np.abs(time - yr))
        
        if np.abs(time[idx] - yr) > 1.0:
            continue
        
        gmsl_at_yr = mc['gmsl_samples'][:, idx]
        var_constrained_per_ssp.append(np.var(gmsl_at_yr))
        median_per_ssp.append(np.median(gmsl_at_yr))
    
    if len(median_per_ssp) == 0:
        continue
    
    # σ²_constrained = average variance across scenarios
    # (parameter uncertainty is roughly the same regardless of scenario)
    sigma2_constrained = np.mean(var_constrained_per_ssp)
    
    # ---- σ²_scenario: variance of medians across SSPs ----
    sigma2_scenario = np.var(median_per_ssp)
    
    # ---- σ²_ice: residual between DOLS range and IPCC full range ----
    # For each SSP, compare DOLS 5-95% range with IPCC 5-95% range
    # The excess IPCC variance is attributed to ice dynamics + other processes
    # not captured by the DOLS semi-empirical model
    
    # IPCC total variance (from 5-95% range, approximated as 1.645σ for each tail)
    ipcc_var_per_ssp = []
    dols_var_per_ssp = []
    
    for sname in scenario_names:
        # IPCC data
        ipcc_key = sname
        if ipcc_key in ipcc_gmsl:
            ipcc_df = ipcc_gmsl[ipcc_key]
            ipcc_row = ipcc_df[np.abs(ipcc_df['decimal_year'] - yr) < 1.0]
            if len(ipcc_row) > 0:
                ipcc_lower = ipcc_row['gmsl_lower'].values[0]
                ipcc_upper = ipcc_row['gmsl_upper'].values[0]
                # 5-95% range → σ ≈ range / (2 × 1.645)
                ipcc_sigma = (ipcc_upper - ipcc_lower) / (2 * 1.645)
                ipcc_var_per_ssp.append(ipcc_sigma**2)
        
        # DOLS variance for this scenario
        mc = mc_results[sname]
        idx = np.argmin(np.abs(mc['time'] - yr))
        if np.abs(mc['time'][idx] - yr) < 1.0:
            dols_var_per_ssp.append(np.var(mc['gmsl_samples'][:, idx]))
    
    # σ²_ice = mean(IPCC variance) - σ²_constrained - σ²_scenario
    # This represents the additional uncertainty in IPCC projections not
    # captured by the semi-empirical model (dominated by ice sheet dynamics)
    if len(ipcc_var_per_ssp) > 0:
        # Average IPCC total variance across SSPs
        sigma2_ipcc_total = np.mean(ipcc_var_per_ssp)
        sigma2_ice = max(0, sigma2_ipcc_total - sigma2_constrained)
    else:
        sigma2_ice = np.nan
    
    # Also store IPCC component-level info at this year
    ais_values = []
    for sname in scenario_names:
        if sname in ipcc_gmsl:
            ipcc_df = ipcc_gmsl[sname]
            row = ipcc_df[np.abs(ipcc_df['decimal_year'] - yr) < 1.0]
            if len(row) > 0 and 'AIS' in row.columns:
                ais_values.append(row['AIS'].values[0])
    
    decomposition.append({
        'year': yr,
        'sigma2_constrained': sigma2_constrained,
        'sigma2_scenario': sigma2_scenario,
        'sigma2_ice': sigma2_ice,
        'sigma2_total': sigma2_constrained + sigma2_scenario + sigma2_ice,
        'median_gmsl_range': median_per_ssp,
        'ais_median': np.mean(ais_values) if ais_values else np.nan,
    })

df_decomp = pd.DataFrame(decomposition)

# Convert to mm for display
for col in ['sigma2_constrained', 'sigma2_scenario', 'sigma2_ice', 'sigma2_total']:
    df_decomp[f'{col}_mm2'] = df_decomp[col] * M_TO_MM**2

# Compute fractional contributions
for col in ['sigma2_constrained', 'sigma2_scenario', 'sigma2_ice']:
    df_decomp[f'frac_{col}'] = df_decomp[col] / df_decomp['sigma2_total']

print('=== Variance Decomposition ===')
print(f'{"Year":>6} {"σ_const (mm)":>14} {"σ_scen (mm)":>14} {"σ_ice (mm)":>14} {"σ_total (mm)":>14} {"f_const":>8} {"f_scen":>8} {"f_ice":>8}')
print('-' * 100)
for _, row in df_decomp.iterrows():
    yr = int(row['year'])
    if yr <= 2150:
        sc = np.sqrt(row['sigma2_constrained_mm2'])
        ss = np.sqrt(row['sigma2_scenario_mm2'])
        si = np.sqrt(row['sigma2_ice_mm2']) if not np.isnan(row['sigma2_ice_mm2']) else 0
        st = np.sqrt(row['sigma2_total_mm2'])
        fc = row['frac_sigma2_constrained']
        fs = row['frac_sigma2_scenario']
        fi = row['frac_sigma2_ice']
        print(f'{yr:>6} {sc:>14.1f} {ss:>14.1f} {si:>14.1f} {st:>14.1f} {fc:>8.1%} {fs:>8.1%} {fi:>8.1%}')

## 5. Hindcast Cross-Validation

To demonstrate that the "predictable" component is genuinely predictable, we calibrate the DOLS model on truncated historical records and evaluate out-of-sample forecast skill.

**Protocol:**
1. Calibrate on data through year $Y_{\text{cut}}$ (using $Y_{\text{cut}}$ ∈ {2000, 2005, 2010})
2. Project forward using *observed* temperatures (not SSP scenarios) from Berkeley Earth
3. Compare projected GMSL against observed GMSL in the withheld period
4. Evaluate: bias, RMSE, coverage of 90% prediction interval

In [ ]:
# ============================================================
# 5a. Hindcast calibration and evaluation
# ============================================================

cutoff_years = [2000, 2005, 2010]
hindcast_results = {}

for cut_yr in cutoff_years:
    # Truncate calibration data
    mask_cal = sl_series.index.year <= cut_yr
    sl_cal = sl_series[mask_cal]
    temp_cal = temp_series[mask_cal]
    
    if len(sl_cal) < 100:  # need reasonable calibration period
        print(f'Skipping cutoff {cut_yr}: insufficient data ({len(sl_cal)} obs)')
        continue
    
    # Calibrate on truncated record
    try:
        result_trunc = calibrate_alpha_dols_quadratic(
            sea_level=sl_cal,
            temperature=temp_cal,
            n_lags=2
        )
    except Exception as e:
        print(f'Calibration failed for cutoff {cut_yr}: {e}')
        continue
    
    # Build coefficients for projection
    coeffs_trunc = np.array([
        result_trunc.dalpha_dT,
        result_trunc.alpha0,
        result_trunc.trend
    ])
    cov_raw_trunc = result_trunc.covariance[:3, :3]
    transform = np.diag([2.0, 1.0, 1.0])
    cov_trunc = transform @ cov_raw_trunc @ transform.T
    
    # Use OBSERVED temperatures for the full period (not SSP projections)
    # Build a temperature DataFrame covering 1950 through end of Berkeley record
    temp_for_proj = df_berkeley[['temperature']].copy()
    temp_for_proj['decimal_year'] = [
        t.year + (t.month - 1) / 12 + (t.day - 1) / 365.25
        for t in temp_for_proj.index
    ]
    # Resample to annual for cleaner projection
    temp_annual = temp_for_proj.resample('YS').mean()
    temp_annual['decimal_year'] = [
        t.year + 0.5 for t in temp_annual.index
    ]
    
    # Run ensemble projection with observed temps
    mc_hind = run_full_ensemble(
        coeffs_trunc, cov_trunc,
        {'observed': temp_annual},
        baseline_year=BASELINE_YEAR,
        n_samples=N_SAMPLES, seed=42
    )[0]
    
    # Extract projected GMSL
    hind_time = mc_hind['observed']['time']
    hind_median = mc_hind['observed']['gmsl_median']
    hind_p5 = mc_hind['observed']['gmsl_p5']
    hind_p95 = mc_hind['observed']['gmsl_p95']
    
    # Get observed GMSL for comparison (Frederikse, annual)
    fred_time = df_frederikse['year'].values if 'year' in df_frederikse.columns else np.array([
        t.year + 0.5 for t in df_frederikse.index
    ])
    fred_gmsl = df_frederikse['gmsl'].values
    
    # Align Frederikse observations to IPCC baseline (2005)
    # Find the value at 2005 in Frederikse and subtract it
    idx_2005_fred = np.argmin(np.abs(fred_time - 2005.0))
    fred_gmsl_rebase = fred_gmsl - fred_gmsl[idx_2005_fred]
    
    # Compute verification metrics for withheld period
    mask_verify = (fred_time > cut_yr) & (fred_time <= 2018)
    if mask_verify.sum() > 0:
        verify_time = fred_time[mask_verify]
        verify_obs = fred_gmsl_rebase[mask_verify]
        
        # Interpolate projection to observation times
        proj_at_obs = np.interp(verify_time, hind_time, hind_median)
        proj_p5_at_obs = np.interp(verify_time, hind_time, hind_p5)
        proj_p95_at_obs = np.interp(verify_time, hind_time, hind_p95)
        
        # Metrics
        bias = np.mean(proj_at_obs - verify_obs) * M_TO_MM
        rmse = np.sqrt(np.mean((proj_at_obs - verify_obs)**2)) * M_TO_MM
        coverage = np.mean((verify_obs >= proj_p5_at_obs) & (verify_obs <= proj_p95_at_obs))
    else:
        bias = rmse = coverage = np.nan
    
    hindcast_results[cut_yr] = {
        'result': result_trunc,
        'coeffs': coeffs_trunc,
        'cov': cov_trunc,
        'time': hind_time,
        'median': hind_median,
        'p5': hind_p5,
        'p95': hind_p95,
        'p17': mc_hind['observed']['gmsl_p17'],
        'p83': mc_hind['observed']['gmsl_p83'],
        'obs_time': fred_time,
        'obs_gmsl': fred_gmsl_rebase,
        'bias_mm': bias,
        'rmse_mm': rmse,
        'coverage_90': coverage,
        'cutoff': cut_yr,
        'n_cal_years': int(cut_yr - fred_time[0]),
    }

print('=== Hindcast Cross-Validation Results ===')
print(f'{"Cutoff":>8} {"Cal years":>10} {"Bias (mm)":>10} {"RMSE (mm)":>11} {"90% Cov":>9}')
print('-' * 55)
for cut_yr, hr in hindcast_results.items():
    print(f'{cut_yr:>8} {hr["n_cal_years"]:>10} {hr["bias_mm"]:>10.2f} {hr["rmse_mm"]:>11.2f} {hr["coverage_90"]:>9.1%}')

In [ ]:
# ============================================================
# 5b. Check coefficient convergence across calibration windows
# ============================================================

print('=== Coefficient Convergence ===')
print(f'{"Cutoff":>8} {"dα/dT (mm/yr/°C²)":>20} {"α₀ (mm/yr/°C)":>18} {"trend (mm/yr)":>16}')
print('-' * 70)
for cut_yr, hr in hindcast_results.items():
    r = hr['result']
    print(f'{cut_yr:>8} {r.dalpha_dT*M_TO_MM:>12.3f} ± {r.dalpha_dT_se*M_TO_MM:.3f} {r.alpha0*M_TO_MM:>10.3f} ± {r.alpha0_se*M_TO_MM:.3f} {r.trend*M_TO_MM:>8.3f} ± {r.trend_se*M_TO_MM:.3f}')
# Full record
print(f'{"Full":>8} {result_full.dalpha_dT*M_TO_MM:>12.3f} ± {result_full.dalpha_dT_se*M_TO_MM:.3f} {result_full.alpha0*M_TO_MM:>10.3f} ± {result_full.alpha0_se*M_TO_MM:.3f} {result_full.trend*M_TO_MM:>8.3f} ± {result_full.trend_se*M_TO_MM:.3f}')

## 6. Hindcast Visualization

In [ ]:
# ============================================================
# 6. Hindcast cross-validation figure
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(14, 4.5), sharey=True)

colors_hind = {2000: '#2166ac', 2005: '#4393c3', 2010: '#92c5de'}

for i, (cut_yr, hr) in enumerate(hindcast_results.items()):
    ax = axes[i]
    
    time = hr['time']
    
    # 90% prediction interval
    ax.fill_between(time, hr['p5'] * M_TO_MM, hr['p95'] * M_TO_MM,
                    alpha=0.15, color=colors_hind[cut_yr], label='90% PI')
    # 66% prediction interval
    ax.fill_between(time, hr['p17'] * M_TO_MM, hr['p83'] * M_TO_MM,
                    alpha=0.3, color=colors_hind[cut_yr], label='66% PI')
    # Median projection
    ax.plot(time, hr['median'] * M_TO_MM, '-', color=colors_hind[cut_yr],
            lw=2, label='Median projection')
    
    # Observations
    mask_cal = hr['obs_time'] <= cut_yr
    mask_ver = hr['obs_time'] > cut_yr
    ax.plot(hr['obs_time'][mask_cal], hr['obs_gmsl'][mask_cal] * M_TO_MM,
            'o', color='#333333', ms=2, alpha=0.5, label='Calibration obs')
    ax.plot(hr['obs_time'][mask_ver], hr['obs_gmsl'][mask_ver] * M_TO_MM,
            's', color='#d62728', ms=3, alpha=0.8, label='Withheld obs')
    
    # Cutoff line
    ax.axvline(cut_yr, color='gray', ls='--', lw=1, alpha=0.7)
    
    # Annotations
    ax.set_title(f'Calibration through {cut_yr}', fontweight='bold')
    ax.set_xlabel('Year')
    ax.set_xlim(1950, 2025)
    
    # Metrics box
    textstr = (f'Bias: {hr["bias_mm"]:.1f} mm\n'
               f'RMSE: {hr["rmse_mm"]:.1f} mm\n'
               f'90% cov: {hr["coverage_90"]:.0%}')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    ax.text(0.03, 0.97, textstr, transform=ax.transAxes, fontsize=8,
            verticalalignment='top', bbox=props)
    
    if i == 0:
        ax.set_ylabel('GMSL relative to 2005 (mm)')
        ax.legend(loc='lower left', fontsize=7)

fig.suptitle('Hindcast Cross-Validation: DOLS Semi-Empirical Model', 
             fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../figures/hindcast_crossvalidation.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: ../figures/hindcast_crossvalidation.png')

## 7. The Key Figure: Predictability Partition

This figure shows how the relative contribution of each uncertainty source evolves with projection horizon. It is the central result of the paper.

In [ ]:
# ============================================================
# 7a. Prepare data for the predictability partition figure
# ============================================================

# Limit to years where we have IPCC data (up to 2150)
df_plot = df_decomp[df_decomp['year'] <= 2150].copy()

# Compute 1-sigma equivalents in mm for each component
df_plot['sigma_constrained_mm'] = np.sqrt(df_plot['sigma2_constrained']) * M_TO_MM
df_plot['sigma_scenario_mm'] = np.sqrt(df_plot['sigma2_scenario']) * M_TO_MM
df_plot['sigma_ice_mm'] = np.sqrt(df_plot['sigma2_ice'].fillna(0)) * M_TO_MM
df_plot['sigma_total_mm'] = np.sqrt(df_plot['sigma2_total']) * M_TO_MM

print('Data prepared for figure.')
print(df_plot[['year', 'sigma_constrained_mm', 'sigma_scenario_mm', 'sigma_ice_mm', 'sigma_total_mm']].to_string(index=False))

In [ ]:
# ============================================================
# 7b. Create the main predictability partition figure
# ============================================================

fig = plt.figure(figsize=(12, 9))
gs = gridspec.GridSpec(2, 2, height_ratios=[1.3, 1], hspace=0.35, wspace=0.3)

# --- Panel A: Stacked area showing variance fractions ---
ax1 = fig.add_subplot(gs[0, 0])

years = df_plot['year'].values
f_const = df_plot['frac_sigma2_constrained'].values
f_scen = df_plot['frac_sigma2_scenario'].values
f_ice = df_plot['frac_sigma2_ice'].fillna(0).values

# Smooth for visual clarity
from scipy.ndimage import uniform_filter1d
# Just use raw values since we have decadal data

ax1.fill_between(years, 0, f_const, 
                 color='#2166ac', alpha=0.8, label='Constrained (DOLS coefficients)')
ax1.fill_between(years, f_const, f_const + f_scen,
                 color='#fdae61', alpha=0.8, label='Scenario (SSP spread)')
ax1.fill_between(years, f_const + f_scen, f_const + f_scen + f_ice,
                 color='#d73027', alpha=0.8, label='Deep uncertainty (ice dynamics)')

ax1.set_ylim(0, 1)
ax1.set_xlim(2020, 2150)
ax1.set_ylabel('Fraction of total variance')
ax1.set_xlabel('Year')
ax1.set_title('(a) Variance fraction by source', fontweight='bold')
ax1.legend(loc='upper left', fontsize=8, framealpha=0.9)
ax1.axvline(2050, color='gray', ls=':', lw=0.8, alpha=0.5)
ax1.axvline(2100, color='gray', ls=':', lw=0.8, alpha=0.5)

# --- Panel B: Absolute 1σ uncertainty by component ---
ax2 = fig.add_subplot(gs[0, 1])

ax2.fill_between(years, 0, df_plot['sigma_constrained_mm'].values,
                 color='#2166ac', alpha=0.6)
ax2.fill_between(years, df_plot['sigma_constrained_mm'].values,
                 df_plot['sigma_constrained_mm'].values + df_plot['sigma_scenario_mm'].values,
                 color='#fdae61', alpha=0.6)
ax2.fill_between(years,
                 df_plot['sigma_constrained_mm'].values + df_plot['sigma_scenario_mm'].values,
                 df_plot['sigma_constrained_mm'].values + df_plot['sigma_scenario_mm'].values + df_plot['sigma_ice_mm'].values,
                 color='#d73027', alpha=0.6)

ax2.set_xlim(2020, 2150)
ax2.set_ylabel('1σ uncertainty (mm)')
ax2.set_xlabel('Year')
ax2.set_title('(b) Absolute uncertainty by source', fontweight='bold')
ax2.axvline(2050, color='gray', ls=':', lw=0.8, alpha=0.5)
ax2.axvline(2100, color='gray', ls=':', lw=0.8, alpha=0.5)

# --- Panel C: DOLS projections vs IPCC ---
ax3 = fig.add_subplot(gs[1, :])

ssp_colors = {
    'SSP1-1.9': '#1b7837', 'SSP1-2.6': '#5aae61',
    'SSP2-4.5': '#fee08b', 'SSP3-7.0': '#fc8d59',
    'SSP5-8.5': '#d73027'
}

for sname in scenario_names:
    mc = mc_results[sname]
    color = ssp_colors[sname]
    
    # DOLS projection
    mask = mc['time'] >= 2005
    t = mc['time'][mask]
    ax3.fill_between(t, mc['gmsl_p5'][mask] * M_TO_MM, mc['gmsl_p95'][mask] * M_TO_MM,
                     alpha=0.12, color=color)
    ax3.plot(t, mc['gmsl_median'][mask] * M_TO_MM, '-', color=color, lw=1.5, label=f'{sname} (DOLS)')
    
    # IPCC AR6 medians and ranges
    ipcc_df = ipcc_gmsl[sname]
    ipcc_t = ipcc_df['decimal_year'].values
    ax3.plot(ipcc_t, ipcc_df['gmsl'].values * M_TO_MM, 's', color=color, ms=4, alpha=0.7)
    ax3.errorbar(ipcc_t, ipcc_df['gmsl'].values * M_TO_MM,
                 yerr=[(ipcc_df['gmsl'].values - ipcc_df['gmsl_lower'].values) * M_TO_MM,
                       (ipcc_df['gmsl_upper'].values - ipcc_df['gmsl'].values) * M_TO_MM],
                 fmt='none', color=color, alpha=0.3, capsize=2)

# Observations
fred_time_plot = df_frederikse['year'].values if 'year' in df_frederikse.columns else np.array([
    t.year + 0.5 for t in df_frederikse.index
])
idx_2005 = np.argmin(np.abs(fred_time_plot - 2005.0))
fred_rebase = (df_frederikse['gmsl'].values - df_frederikse['gmsl'].values[idx_2005]) * M_TO_MM
ax3.plot(fred_time_plot, fred_rebase, 'k-', lw=1.5, alpha=0.6, label='Observed (Frederikse)')

ax3.set_xlim(1990, 2150)
ax3.set_ylim(-50, 1000)
ax3.set_xlabel('Year')
ax3.set_ylabel('GMSL relative to 2005 (mm)')
ax3.set_title('(c) DOLS semi-empirical projections vs IPCC AR6 (lines = DOLS, squares = IPCC)', fontweight='bold')
ax3.legend(ncol=3, fontsize=7, loc='upper left', framealpha=0.9)
ax3.axvline(2050, color='gray', ls=':', lw=0.8, alpha=0.5)
ax3.axvline(2100, color='gray', ls=':', lw=0.8, alpha=0.5)

fig.suptitle('Predictability of 21st-Century Sea-Level Rise', 
             fontsize=14, fontweight='bold', y=1.01)

plt.savefig('../figures/predictability_partition.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: ../figures/predictability_partition.png')

## 8. IPCC Component Breakdown — Where Deep Uncertainty Lives

Examine the IPCC AR6 component-level projections to identify which processes drive the deep uncertainty residual. The AIS contribution is nearly SSP-independent, confirming it represents a fundamentally different type of uncertainty.

In [ ]:
# ============================================================
# 8. IPCC component analysis
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Panel A: Component contributions by SSP at 2100
ax = axes[0]
components = ['oceandynamics', 'AIS', 'GIS', 'glaciers', 'landwaterstorage']
comp_labels = ['Ocean\ndynamics', 'AIS', 'GIS', 'Glaciers', 'Land water\nstorage']
comp_colors = ['#2166ac', '#d73027', '#f4a582', '#92c5de', '#b2abd2']

x = np.arange(len(scenario_names))
width = 0.15

for j, (comp, clabel, color) in enumerate(zip(components, comp_labels, comp_colors)):
    vals = []
    for sname in scenario_names:
        row_2100 = ipcc_gmsl[sname][np.abs(ipcc_gmsl[sname]['decimal_year'] - 2100) < 1]
        vals.append(row_2100[comp].values[0] * M_TO_MM if len(row_2100) > 0 else 0)
    ax.bar(x + j * width, vals, width, label=clabel, color=color, alpha=0.85)

ax.set_xticks(x + 2 * width)
ax.set_xticklabels([s.replace('SSP', '') for s in scenario_names], fontsize=9)
ax.set_ylabel('Contribution at 2100 (mm rel. to 2005)')
ax.set_title('(a) IPCC AR6 GMSL components at 2100', fontweight='bold')
ax.legend(fontsize=8, ncol=2)

# Panel B: AIS contribution over time — nearly SSP-independent
ax = axes[1]
for sname in scenario_names:
    ipcc_df = ipcc_gmsl[sname]
    mask = ipcc_df['decimal_year'] <= 2150
    ax.plot(ipcc_df.loc[mask, 'decimal_year'], ipcc_df.loc[mask, 'AIS'] * M_TO_MM,
            'o-', color=ssp_colors[sname], ms=4, label=sname)

ax.set_xlabel('Year')
ax.set_ylabel('AIS median contribution (mm)')
ax.set_title('(b) AIS contribution: nearly SSP-independent', fontweight='bold')
ax.legend(fontsize=8)

plt.tight_layout()
plt.savefig('../figures/ipcc_components.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: ../figures/ipcc_components.png')

## 9. Summary Statistics and Key Results

In [ ]:
# ============================================================
# 9. Summary table of key results
# ============================================================

print('=' * 80)
print('SUMMARY: Predictability of 21st-Century Sea-Level Rise')
print('=' * 80)

print('\n--- DOLS Calibration (Full Record) ---')
print(f'  dα/dT = {result_full.dalpha_dT*M_TO_MM:.3f} ± {result_full.dalpha_dT_se*M_TO_MM:.3f} mm/yr/°C²')
print(f'  α₀    = {result_full.alpha0*M_TO_MM:.3f} ± {result_full.alpha0_se*M_TO_MM:.3f} mm/yr/°C')
print(f'  trend = {result_full.trend*M_TO_MM:.3f} ± {result_full.trend_se*M_TO_MM:.3f} mm/yr')
print(f'  R²    = {result_full.r2:.4f}')

print('\n--- Variance Decomposition at Key Horizons ---')
for yr in [2050, 2100, 2150]:
    row = df_decomp[df_decomp['year'] == yr]
    if len(row) > 0:
        row = row.iloc[0]
        print(f'\n  {yr}:')
        print(f'    Constrained (coefficient):  {row["frac_sigma2_constrained"]:.0%} of total variance')
        print(f'    Scenario (SSP spread):      {row["frac_sigma2_scenario"]:.0%} of total variance')
        print(f'    Deep uncertainty (ice):      {row["frac_sigma2_ice"]:.0%} of total variance')
        print(f'    Total 1σ:                   {np.sqrt(row["sigma2_total"])*M_TO_MM:.0f} mm')

print('\n--- Hindcast Skill ---')
for cut_yr, hr in hindcast_results.items():
    print(f'  Cal through {cut_yr}: bias={hr["bias_mm"]:.1f} mm, RMSE={hr["rmse_mm"]:.1f} mm, 90% coverage={hr["coverage_90"]:.0%}')

print('\n--- Key Messages ---')
row_2050 = df_decomp[df_decomp['year'] == 2050].iloc[0]
row_2100 = df_decomp[df_decomp['year'] == 2100].iloc[0]
print(f'  1. Near-term SLR (to 2050) is dominated by constrained uncertainty')
print(f'     ({row_2050["frac_sigma2_constrained"]:.0%} of variance from observable-constrained parameters).')
print(f'  2. By 2100, deep uncertainty from ice sheet dynamics contributes')
print(f'     {row_2100["frac_sigma2_ice"]:.0%} of total variance — this cannot be reduced by')
print(f'     longer global temperature or sea level records.')
print(f'  3. The DOLS model demonstrates genuine out-of-sample predictive skill')
print(f'     (RMSE < {max(hr["rmse_mm"] for hr in hindcast_results.values()):.0f} mm across all hindcast windows).')
print(f'  4. AIS contribution at 2100 ({row_2100["ais_median"]*M_TO_MM:.0f} mm) is nearly SSP-independent,')
print(f'     confirming it represents a structurally different type of uncertainty.')

In [ ]:
# ============================================================
# 10. Supplementary: coefficient stability plot
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(13, 4))

param_names = ['dα/dT', 'α₀', 'trend']
param_units = ['mm/yr/°C²', 'mm/yr/°C', 'mm/yr']

for j, (pname, punit) in enumerate(zip(param_names, param_units)):
    ax = axes[j]
    
    cut_years_list = sorted(hindcast_results.keys())
    x_positions = list(range(len(cut_years_list))) + [len(cut_years_list)]
    x_labels = [str(yr) for yr in cut_years_list] + ['Full']
    
    vals = []
    errs = []
    for cut_yr in cut_years_list:
        r = hindcast_results[cut_yr]['result']
        if j == 0:
            vals.append(r.dalpha_dT * M_TO_MM)
            errs.append(r.dalpha_dT_se * M_TO_MM)
        elif j == 1:
            vals.append(r.alpha0 * M_TO_MM)
            errs.append(r.alpha0_se * M_TO_MM)
        else:
            vals.append(r.trend * M_TO_MM)
            errs.append(r.trend_se * M_TO_MM)
    
    # Full record
    if j == 0:
        vals.append(result_full.dalpha_dT * M_TO_MM)
        errs.append(result_full.dalpha_dT_se * M_TO_MM)
    elif j == 1:
        vals.append(result_full.alpha0 * M_TO_MM)
        errs.append(result_full.alpha0_se * M_TO_MM)
    else:
        vals.append(result_full.trend * M_TO_MM)
        errs.append(result_full.trend_se * M_TO_MM)
    
    ax.errorbar(x_positions, vals, yerr=[e * 1.96 for e in errs], 
                fmt='o', color='#2166ac', capsize=5, ms=7)
    
    # Highlight full-record value
    ax.axhline(vals[-1], color='#d73027', ls='--', lw=1, alpha=0.5)
    
    ax.set_xticks(x_positions)
    ax.set_xticklabels(x_labels)
    ax.set_xlabel('Calibration window end')
    ax.set_ylabel(f'{pname} ({punit})')
    ax.set_title(f'{pname}', fontweight='bold')

fig.suptitle('Coefficient Stability Across Calibration Windows (95% CI)',
             fontsize=12, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../figures/coefficient_stability.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: ../figures/coefficient_stability.png')